In [1]:
# Code Template for the New Notebook to Create Run Charts

# Importing necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import numpy as np
import ipywidgets as widgets
from ipywidgets import interact_manual



In [2]:
def calculate_average_scores(df, frequency):
    """
    Calculate average scores based on the given frequency.
    Weekly frequency starts every Monday, Monthly on the 1st of each month, 
    and Quarterly starting from the 1st of January, April, July, and October.
    """
    # Ensure 'Date' is a datetime type
    df['Date'] = pd.to_datetime(df['Date'])

    # Group by 'Unit', 'Type', and resample based on frequency, then calculate mean of 'Average Score'
    if frequency == 'Weekly':
        resampled_df = df.set_index('Date').groupby(['Unit', 'Type']).resample('W-MON')['Average Score'].mean().reset_index()
    elif frequency == 'Monthly':
        resampled_df = df.set_index('Date').groupby(['Unit', 'Type']).resample('MS')['Average Score'].mean().reset_index()
    elif frequency == 'Quarterly':
        resampled_df = df.set_index('Date').groupby(['Unit', 'Type']).resample('QS-JAN')['Average Score'].mean().reset_index()

    # Convert the average scores from out of 100 to out of 5
    resampled_df['Average Score'] = resampled_df['Average Score'] / 20

    return resampled_df

In [3]:
def plot_run_charts(df, frequency):
    """
    Plot run charts for each unit and an overall combined chart.
    The combined chart will display the average scores aggregated from all units for 'Patient' and 'Visitor'.
    """
    # Calculate average scores based on frequency
    avg_scores = calculate_average_scores(df, frequency)

    # List of unique units
    units = df['Unit'].unique().tolist()

    # Plot for each unit
    for unit in units:
        plot_individual_run_chart(unit, avg_scores, frequency)

    # Plot the combined chart
    plot_combined_run_chart(avg_scores, frequency)

def plot_individual_run_chart(unit, avg_scores, frequency):
    """
    Plot run chart for an individual unit.
    """
    plt.figure(figsize=(12, 6))
    plt.title(f"Run Chart for {unit} - {frequency} Frequency")
    
    # Filtering data for the current unit
    unit_df = avg_scores[avg_scores['Unit'] == unit]

    # Plotting logic for individual unit
    plot_run_chart_logic(unit_df)

def plot_combined_run_chart(avg_scores, frequency):
    """
    Plot the combined run chart for all units.
    """
    plt.figure(figsize=(12, 6))
    plt.title(f"Combined Run Chart for All Units - {frequency} Frequency")

    # Calculating the overall average for 'Patient' and 'Visitor' across all units
    combined_df = avg_scores.groupby(['Date', 'Type'])['Average Score'].mean().reset_index()

    # Plotting logic for combined chart
    plot_run_chart_logic(combined_df)

def plot_run_chart_logic(df):
    """
    Shared plotting logic for run charts.
    """
    # Creating separate dataframes for 'Patient' and 'Visitor'
    patient_df = df[df['Type'] == 'Patient']
    visitor_df = df[df['Type'] == 'Visitor']

    patient_color = '#00727C'
    visitor_color = '#9ED9D1'
    
    # Plotting 'Patient' scores
    plt.plot(patient_df['Date'], patient_df['Average Score'], label='Patient', marker='o', linestyle='-', color=patient_color)

    # Plotting 'Visitor' scores if available
    if not visitor_df.empty:
        plt.plot(visitor_df['Date'], visitor_df['Average Score'], label='Visitor', marker='x', linestyle='-', color=visitor_color)

    # Formatting the x-axis to display the month
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator())

    plt.xlabel('Date')
    plt.ylabel('Average Score')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.ylim(0,5.3)
    plt.show()

# Usage: Call plot_run_charts with the DataFrame and desired frequency
# plot_run_charts(df, 'Weekly') for example


In [4]:
# Interactive Widgets
frequency_widget = widgets.Dropdown(
    options=['Weekly', 'Monthly', 'Quarterly'],
    value='Weekly',
    description='Frequency:',
)

In [5]:
# Function to update visualizations
def update_visualizations(frequency):
    # Load and preprocess the data (to be implemented)
    df = pd.read_csv('cleaned_survey.csv')

    # Plot run charts
    plot_run_charts(df, frequency)



In [6]:
# Display the interactive widget
interact_manual(update_visualizations, frequency=frequency_widget)


interactive(children=(Dropdown(description='Frequency:', options=('Weekly', 'Monthly', 'Quarterly'), value='We…

<function __main__.update_visualizations(frequency)>